In [1]:
import gc

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import joblib
import random
import os
import lightgbm as lgb

In [3]:
data_path = "/home/data/m5"

calender = pd.read_csv(os.path.join(data_path,'calendar.csv'))
sales_train_evalutaion = pd.read_csv(os.path.join(data_path,'sales_train_evaluation.csv'))
sales_train_validation = pd.read_csv(os.path.join(data_path,'sales_train_validation.csv'))
sample_submission = pd.read_csv(os.path.join(data_path,'sample_submission.csv'))
sell_prices = pd.read_csv(os.path.join(data_path,'sell_prices.csv'))

## Preparation for calender data

In [4]:
calender.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [5]:
sales_train_evalutaion.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [6]:
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# adding zero sales for next 28 days, as we are going to predict the same
for day in range(1942, 1970):
    col = 'd_' + str(day)
    sales_train_evalutaion[col] = 0
    sales_train_evalutaion[col] = sales_train_evalutaion[col].astype(np.int16)

In [8]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

In [9]:
%%time
print("Downcasting the data")
sales_train_evalutaion = downcast(sales_train_evalutaion)
sell_prices = downcast(sell_prices)
calender = downcast(calender)

Downcasting the data
CPU times: user 49.5 s, sys: 1min 48s, total: 2min 37s
Wall time: 2min 37s


In [10]:
# from statsmodels.tsa.arima.model import ARIMA

In [11]:
# import copy

In [12]:
# Load the data
# # Assuming sales_data is a DataFrame where rows represent dates, columns represent items, and values represent sales
# sales_data = copy.deepcopy(sales_train_evalutaion)

In [13]:
# forecasts = pd.DataFrame(index=sales_data.index, columns=sales_data.columns)

In [14]:
# for col in sales_data.columns:
#     model = ARIMA(sales_data[col], order=(5,1,0))
#     model_fit = model.fit(disp=0)
#     forecasts[col] = model_fit.forecast(steps=30)

# # Aggregate the forecasts to higher levels
# # Assuming hierarchies is a dict where keys represent higher-level series, values represent lists of lower-level series
# aggregated_forecasts = {}
# for key, values in hierarchies.items():
#     aggregated_forecasts[key] = forecasts[values].sum(axis=1)

In [15]:
%%time
print("Melting data")
df = pd.melt(frame=sales_train_evalutaion, 
             id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
             var_name="d", value_name="sold")

Melting data
CPU times: user 4.63 s, sys: 2.6 s, total: 7.23 s
Wall time: 7.23 s


In [16]:
%%time
print("Merging data")
df = pd.merge(left=df, right=calender, how="left", on="d")
df = pd.merge(left=df, right=sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

Merging data
CPU times: user 22.8 s, sys: 7.89 s, total: 30.7 s
Wall time: 30.7 s


In [17]:
%%time
print("Implement features")
#Calculate the SNAP (Supplemental Nutrition Assistance Program) day for each state
df["snap"] = df["snap_CA"] + df["snap_TX"] + df["snap_WI"]
df["snap"] = np.where(df["snap"] >= 1, 1, 0).astype(np.int8)

# Apply int for day column
df["d"] = df["d"].str[2:].astype(np.int16)

# Process NaN value
df["sell_price"] = df['sell_price'].fillna(df.groupby('id')['sell_price'].transform('median'))

# Is it a weekend
df["weekend"] = np.where(df["wday"] < 3, 1, 0).astype(np.int8)

# Drop unnecessary columns
df = df.drop(["date", "weekday", "wm_yr_wk", "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI"], axis=1)

Implement features
CPU times: user 22.9 s, sys: 4.19 s, total: 27.1 s
Wall time: 27.1 s


In [18]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wday,month,year,event_name_1,event_type_1,sell_price,snap,weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,8.257812,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,3.970703,0,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,2.970703,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,4.640625,0,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,2011,NaN,NaN,2.980469,0,1


In [19]:
# Label Encoder
print("Label Encoding")
d_id = dict(zip(df["id"].cat.codes, df["id"]))
d_store = dict(zip(df["store_id"].cat.codes, df["store_id"]))
df["id"] = df["id"].cat.codes
df["item_id"] = df["item_id"].cat.codes
df["dept_id"] = df["dept_id"].cat.codes
df["cat_id"] = df["cat_id"].cat.codes
df["store_id"] = df["store_id"].cat.codes
df["state_id"] = df["state_id"].cat.codes
df["event_name_1"] = df["event_name_1"].cat.codes
df["event_type_1"] = df["event_type_1"].cat.codes

Label Encoding


In [20]:
%%time
print("Calulating Lags and Rolling mean")
# Lags must be > 28
lags = [29,30,31,32,33,34,35,40,55,60,65,180]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
    
df['rolling_mean_7']   = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(7).mean())
df['rolling_mean_14']   = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(14).mean())
df['rolling_mean_30']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(30).mean())
df['rolling_mean_60']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(60).mean())
df['rolling_mean_180']  = df.groupby(['id'])['sold'].transform(lambda x: x.shift(28).rolling(180).mean())

Calulating Lags and Rolling mean
CPU times: user 3min 41s, sys: 1min 3s, total: 4min 45s
Wall time: 4min 45s


In [21]:
df = df[df["d"] > 28+180]

In [22]:
# Save dataframe
df.to_pickle('data.pkl')
del df

In [23]:
gc.collect()

0

In [24]:
gc.collect()

0

In [25]:
%%time
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]

CPU times: user 197 ms, sys: 2.91 s, total: 3.11 s
Wall time: 3.11 s


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853720 entries, 59181090 to 60034809
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   id      853720 non-null  int16
 1   d       853720 non-null  int16
 2   sold    853720 non-null  int16
dtypes: int16(3)
memory usage: 11.4 MB


In [27]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wday,month,...,sold_lag_40,sold_lag_55,sold_lag_60,sold_lag_65,sold_lag_180,rolling_mean_7,rolling_mean_14,rolling_mean_30,rolling_mean_60,rolling_mean_180
6341920,14370,1437,3,1,0,0,209,0,6,8,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6341921,14380,1438,3,1,0,0,209,0,6,8,...,0.0,1.0,0.0,1.0,0.0,0.142857,0.071429,0.200000,0.133333,0.044444
6341922,14390,1439,3,1,0,0,209,0,6,8,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6341923,14400,1440,3,1,0,0,209,1,6,8,...,4.0,3.0,5.0,1.0,0.0,1.428571,1.357143,1.700000,1.333333,0.844444
6341924,14410,1441,3,1,0,0,209,0,6,8,...,0.0,0.0,0.0,7.0,0.0,0.000000,0.000000,0.066667,0.550000,0.355556


In [28]:
best_params_store = (
{'CA_1': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'CA_2': {'colsample_bytree': 0.9,
  'learning_rate': 0.30000000000000004,
  'max_depth': 8.0,
  'min_child_weight': 200.0,
  'num_leaves': 200.0,
  'subsample': 0.8},
 'CA_3': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 300.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'CA_4': {'colsample_bytree': 0.9,
  'learning_rate': 0.30000000000000004,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'TX_1': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 1.0},
 'TX_2': {'colsample_bytree': 1.0,
  'learning_rate': 0.30000000000000004,
  'max_depth': 8.0,
  'min_child_weight': 300.0,
  'num_leaves': 200.0,
  'subsample': 0.9},
 'TX_3': {'colsample_bytree': 0.8,
  'learning_rate': 0.2,
  'max_depth': 7.0,
  'min_child_weight': 300.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'WI_1': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 400.0,
  'num_leaves': 250.0,
  'subsample': 0.8},
 'WI_2': {'colsample_bytree': 1.0,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 200.0,
  'num_leaves': 250.0,
  'subsample': 0.9},
 'WI_3': {'colsample_bytree': 0.9,
  'learning_rate': 0.2,
  'max_depth': 8.0,
  'min_child_weight': 400.0,
  'num_leaves': 250.0,
  'subsample': 0.9}})

In [29]:
def cross_validation(data, valid_first_day, params, day_start, d_store=d_store, sample_sub=sample_submission):
    
    valid = data[(data['d']>=valid_first_day) & (data['d']<valid_first_day+28)][['id','d','sold']]
    print(f"Valid first day {valid_first_day} predicting")
    for i in range(10): 
        # Forecast cho từng store 
        df = data[data["store_id"] == i]

        #Create train set
        X_train, y_train = df[(df['d']>=day_start) & (df['d']<valid_first_day)].drop('sold',axis=1), df[(df['d']>=day_start) & (df['d']<valid_first_day)]['sold']
        train_sets = lgb.Dataset(X_train, y_train)
        X_valid, y_valid = df[(df['d']>=valid_first_day) & (df['d']<valid_first_day+28)].drop('sold',axis=1), df[(df['d']>=valid_first_day) & (df['d']<valid_first_day+28)]['sold']
        valid_sets = lgb.Dataset(X_valid, y_valid)

        model = lgb.train(params={'objective' : 'tweedie',
                                  'force_row_wise': True,
                                  'verbose': -1,
                                  'n_estimators': 1000,
                                  'learning_rate':params[d_store[i]]["learning_rate"],
                                  'subsample': params[d_store[i]]["subsample"],
                                  'colsample_bytree':params[d_store[i]]["colsample_bytree"],
                                  'min_child_weight':params[d_store[i]]["min_child_weight"],
                                  'max_depth':np.int16(params[d_store[i]]["max_depth"]),
                                  'num_leaves':np.int16(params[d_store[i]]["num_leaves"])},
                                
                      train_set=train_sets, 
                      valid_sets=valid_sets,
                      verbose_eval=False,
                      early_stopping_rounds=50)

        pred_val = model.predict(X_valid)
        valid.loc[X_valid.index, "sold"] = pred_val

    valid["id"] = valid["id"].map(d_id)
    valid = valid.pivot(index="id", columns="d", values="sold").reset_index()
    valid["id"] = valid["id"].str.replace("evaluation", "validation")
    
    sample_sub = sample_sub[["id"]]

    f_col = [f"F{i}" for i in range(1,29)]
    f_col.insert(0, "id")
    
    print(f"Valid testset from day {valid.columns[1]} to day {valid.columns[-1]}")
    
    out_val = pd.merge(left=sample_sub[:30490], right=valid, on="id")
    out_val.columns=f_col
    
    return out_val

def avg_rmsse_score(out_val, train_eva, valid_first_day, day_start):
    
    print(f"Scoring from {valid_first_day} to {valid_first_day+28-1}")
    print(f"Naive first day {day_start}")
    
    days_train = [i for i in range(day_start, valid_first_day)]
    days_valid = [i for i in range(valid_first_day, valid_first_day+28)]
    
    naive_predict = np.array(train_eva[days_train].drop(valid_first_day-1, axis=1)).astype(np.int32)
    y_true_naive = np.array(train_eva[days_train].drop(day_start, axis=1)).astype(np.int32)
    naive_mse = np.mean((naive_predict - y_true_naive) ** 2, axis=1)
    
    model_pred = np.array(out_val.iloc[:, 1:])
    y_true_model = np.array(train_eva[days_valid])
    model_mse = np.mean((model_pred - y_true_model) ** 2, axis=1)
    
    avg_rmsse = np.sqrt(model_mse / naive_mse).mean()
    
    return avg_rmsse


In [30]:
sales_train_evalutaion.columns = list(sales_train_evalutaion.columns[:6]) + [i for i in range(1, 1970)]

In [31]:
valid_first_days_list = [1858, 1886, 1914]
cv_score = dict()

In [33]:
%%time
valid_first_days_list = [1858, 1886, 1914]
day_start = 209
day_start_naive = 1
cv_score = dict()
for i in valid_first_days_list:
    out_df_cv = cross_validation(data=data, valid_first_day=i, day_start=day_start, params=best_params_store)
    cv_score[i] = avg_rmsse_score(out_val=out_df_cv, train_eva=sales_train_evalutaion, valid_first_day=i, day_start=day_start_naive)
    day_start += 28
    day_start_naive += 28

Valid first day 1858 predicting
Valid testset from day 1858 to day 1885
Scoring from 1858 to 1885
Naive first day 1
Valid first day 1886 predicting
Valid testset from day 1886 to day 1913
Scoring from 1886 to 1913
Naive first day 29
Valid first day 1914 predicting
Valid testset from day 1914 to day 1941
Scoring from 1914 to 1941
Naive first day 57
CPU times: user 1h 13min 24s, sys: 1min 5s, total: 1h 14min 29s
Wall time: 12min 5s


In [34]:
for i in valid_first_days_list:
    print(f"Score {i}", cv_score[i])
    
print("CV score", np.mean(list(cv_score.values())))

Score 1858 0.9123814090623421
Score 1886 0.897589270152756
Score 1914 0.9106487788471872
CV score 0.9068731526874285
